In [25]:
import geopandas as gpd
import pandas as pd
import folium
from IPython.display import display
import unicodedata

In [3]:
df = pd.read_csv('../data/consolidated_properties/data_consolidada_RM.csv', index_col=0)
df = df.loc[df.proyecto==False]
filtrado = df[(df['comuna'] == 'la-reina') & (df['dormitorios_clean'] == 1.0)]

In [4]:
#!wget https://www.bcn.cl/siit/obtienearchivo?id=repositorio/10221/10396/1/division_comunal.zip

# df = gpd.read_file('../data/maps/division_comunal/division_comunal.shp')
# df_rm = df[(df.NOM_REG=='Región Metropolitana de Santiago')&(df.NOM_PROV=='Santiago')]
# df_rm.to_file("../data/maps/division_comunal.geojson", driver="GeoJSON")

In [33]:
comunas_geo = gpd.read_file("../data/maps/division_comunal.geojson")
comunas_geo['NOM_COM'] = comunas_geo['NOM_COM'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8'))
comunas_geo.head(2)

,NOM_REG,NOM_PROV,NOM_COM,SHAPE_LENG,DIS_ELEC,CIR_SENA,COD_COMUNA,SHAPE_Le_1,SHAPE_Area,geometry
0,Región Metropolitana de Santiago,Santiago,Independencia,11488.695747,19,7,1310,11718.687086,7.514746e+06,"POLYGON ((345884.003 6302609.684, 345884.8 630..."
1,Región Metropolitana de Santiago,Santiago,La Florida,49696.426793,26,8,1311,49621.402175,7.103733e+07,"POLYGON ((366277.23 6293233.79, 366389.722 629..."


In [34]:
df = pd.read_csv('../data/consolidated_properties/data_consolidada_RM.csv', index_col=0)
df['comuna'] = df['comuna'].str.split('-').apply(lambda x: ' '.join([word.capitalize() for word in x]))
df['comuna'] = df['comuna'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8'))
df_comuna = df.groupby(['comuna']).agg({'price_clean_uf': 'mean'}).reset_index()
df_comuna.head(2)

,comuna,price_clean_uf
0,Cerrillos,20.766596
1,Colina,35.264000


In [35]:
df_comuna_geo = comunas_geo.merge(df_comuna, how='inner', left_on='NOM_COM', right_on='comuna')

In [54]:
comunas_geo.NOM_COM.unique().shape, comunas_geo.shape

((32,), (32, 10))

In [37]:
m = folium.Map(location=[-33.45, -70.65], zoom_start=10)

folium.Choropleth(
    geo_data=df_comuna_geo,
    data=df_comuna_geo,
    columns=['comuna', 'price_clean_uf'],
    key_on='feature.properties.NOM_COM',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Variable por comuna'
).add_to(m)

# m.save('mapa_santiago.html')
# display(m)

In [43]:
# df_comuna_geo.plot()
# df_comuna_geo.explore()